In [1]:
import json
import pandas as pd

paper_filename = '../data/paper.csv'
scholar_filename = '../data/scholar.csv'
output_statistic_file = '../data/statistic.json'
output_rank_file = '../data/rank.csv'

## Data from Paper Table

In [2]:
df_paper = pd.read_csv(paper_filename, sep=',', header=0)
df_paper = df_paper.sort_values(['year', 'booktitle', 'title'], ascending=False)
display(df_paper)

,year,type,author,title,field,tag,booktitle,abbr,vol,no,pages,doi
0,2025,article,"Yunlong Sheng, Xuye Zhuang, Jiancheng Yin, Bow...",A combinatorial test case prioritization metho...,Optimization,NaN,The Journal of Supercomputing,NaN,81,8,997,10.1007/S11227-025-07459-5
1,2025,article,"Eduard Baranov, Axel Legay",Baital: Sampling configurable systems with hig...,Generation,NaN,Science of Computer Programming,NaN,240,NaN,103209,10.1016/J.SCICO.2024.103209
2,2025,article,"Andrea Bombarda, Angelo Gargantini",On the Completion of Partial Combinatorial Tes...,Generation,NaN,SN Computer Science,NaN,6,4,383,10.1007/S42979-025-03937-Y
3,2025,article,"Pierre Martou, Benoît Duhoux, Kim Mens, Axel L...",Combinatorial transition testing in dynamicall...,Application,NaN,Journal of Systems and Software,JSS,221,NaN,112260,10.1016/J.JSS.2024.112260
4,2025,article,"Kambiz Nezami Balouchi, Julien Mercier, Robert...",An exploratory empirical eye-tracker study of ...,Application,NaN,Journal of Systems and Software,JSS,220,NaN,112261,10.1016/J.JSS.2024.112261
...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1989,article,"Charles J. Colbourn, Paul C. van Oorschot",Applications of combinatorial designs in compu...,Application,Survey,ACM Computing Surveys,CSUR,21,2,223-250,10.1145/66443.66446
1096,1988,article,"Gadiel Seroussi, Nader H. Bshouty",Vector Sets for Exhaustive Testing of Logic Ci...,Application,NaN,IEEE Transaction on Information Theory,NaN,34,3,513-522,10.1109/18.6031
1097,1987,inproceedings,Keizo Tatsumi,Test Case Design Support System,Application,Constraint,International Conference on Quality Control,NaN,NaN,NaN,615-620,NaN
1098,1987,inproceedings,"Keizo Tatsumi, S. Watanabe, Y. Takeuchi, H. Sh...",Conceptual support for test case design,Application,Constraint,"International Computers, Software & Applicatio...",COMPSAC,NaN,NaN,285-290,NaN


In [3]:
# number of publications
publication_data = df_paper.groupby('year').size().to_frame('number')
publication_data['cumulative'] = publication_data['number'].cumsum()
publication_data = publication_data.reset_index()
display(publication_data)

,year,number,cumulative
0,1985,1,1
1,1987,2,3
2,1988,1,4
3,1989,1,5
4,1992,1,6
5,1994,3,9
6,1995,1,10
7,1996,3,13
8,1997,2,15
9,1998,6,21


In [4]:
# distribution of topics
distribution_data = df_paper.groupby('field').size().to_frame('count').reset_index()
distribution_data = distribution_data.sort_values('count', ascending=False)
distribution_data

,field,count
3,Generation,469
0,Application,343
2,Evaluation,89
5,Optimization,71
4,Model,52
1,Diagnosis,46
6,Other,28


In [5]:
# number of publication of each topic per year
year = publication_data['year'].values.tolist()
field = distribution_data['field'].values.tolist()

# create a new data frame for all combinations of <year, field>
full_index = pd.MultiIndex.from_product([year, field], names=['year', 'field'])
full_df = pd.DataFrame(index=full_index).reset_index()

# calculate number of papers for each combiantion of <year, field>
temp = df_paper.groupby(['year', 'field']).size().to_frame('number').reset_index()

# merge
topic_data = full_df.merge(temp, on=['year', 'field'], how='left').fillna(0)
topic_data['number'] = topic_data['number'].astype(int) 

display(topic_data)

,year,field,number
0,1985,Generation,0
1,1985,Application,1
2,1985,Evaluation,0
3,1985,Optimization,0
4,1985,Model,0
...,...,...,...
254,2025,Evaluation,2
255,2025,Optimization,2
256,2025,Model,1
257,2025,Diagnosis,0


### Final Data

Index page:
- `cumulative-2000`: cumulative number of publications, from 2000 to present (bar chart)
- `cumulative`: cumulative number of publications (bar chart)
- `distribution`: distribution of topics (pie chart)

Statictics chart page:
- `annual`: annual number of publications (line chart)
- `topic`: annual number of publications per topic (bar chart, stacked)

In [6]:
data = {
    'cumulative-2000': {},   # {year = [], value = []}
    'cumulative': {},        # {year = [], value = []}
    'annual': {},       # {year = [], value = []}
    'distribution': {}, # {field = [], count = []}
    'topic': {}         # {field_1 = [], field_2 = [], ...}       
}

year = publication_data['year'].values.tolist()
number = publication_data['number'].values.tolist()
cumulative = publication_data['cumulative'].values.tolist()

# cumulative data, from 2000 to now
index = year.index(2000)
data['cumulative-2000']['year'] = year[index:]
data['cumulative-2000']['value'] = cumulative[index:]

# cumulative data
data['cumulative']['year'] = year
data['cumulative']['value'] = cumulative

# annual data
data['annual']['year'] = year
data['annual']['value'] = number

# distribution data
fields = distribution_data['field'].values.tolist()
data['distribution']['fields'] = fields
data['distribution']['count'] = distribution_data['count'].values.tolist()

# topic data
for field in fields:
    subset = topic_data[topic_data['field'] == field]
    data['topic'][field] = subset['number'].tolist()
    assert(len(data['topic'][field]) == len(year))

In [7]:
# write data to json file
with open(output_statistic_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## Data from Scholar Table

In [8]:
scholar = pd.read_csv(scholar_filename, sep=',', header=0)
display(scholar)

,id,name,institution,country,homepage,TSE,TOSEM,EMSE,JSS,IST,ICSE,FSE,ASE,ISSTA,SANER,ISSRE,ICST,ICSTW
0,1,Kenneth Lausdahl,Aarhus University,Denmark,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Peter Gorm Larsen,Aarhus University,Denmark,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Erik Carlson,ABB,Norway,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Jan Endresen,ABB,Norway,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Tormod Wien,ABB,Norway,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,1409,Sabrina Böhm,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
1407,1410,Davar Giveki,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
1408,1411,Amir Rastegarnia,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
1409,1412,Ali Farzamnia,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
weight = {'TSE': 3, 'TOSEM': 3, 'EMSE': 2, 'JSS': 2, 'IST': 2, 'ICSE': 3, 'FSE': 3, 'ASE': 3, 'ISSTA': 3, 
          'SANER': 2, 'ISSRE': 2, 'ICST': 2, 'ICSTW': 1}

def calc_score(row):
    score = 0
    for each in weight:
        score += row[each]
    return score

scholar['score'] = scholar.apply(calc_score, axis=1)
scholar = scholar.sort_values(['score', 'name'], ascending=False)
scholar = scholar[scholar['score'] >= 4]
scholar.insert(0, 'rank', scholar['score'].rank(method='min', ascending=False).astype(int))

display(scholar)

,rank,id,name,institution,country,homepage,TSE,TOSEM,EMSE,JSS,IST,ICSE,FSE,ASE,ISSTA,SANER,ISSRE,ICST,ICSTW,score
520,1,521,Yu Lei,University of Texas at Arlington,United States,http://ranger.uta.edu/~ylei/,8,0,0,0,0,0,0,2,0,0,1,3,31,45
270,1,271,Raghu N. Kacker,National Institute of Standards and Technology...,United States,http://math.nist.gov/~RKacker/,2,0,2,0,0,0,0,1,0,0,1,3,36,45
264,3,265,D. Richard Kuhn,National Institute of Standards and Technology...,United States,http://csrc.nist.gov/staff/Kuhn/kuhn_rick.html,3,0,2,0,0,0,0,1,0,0,1,3,34,44
478,4,479,Myra B. Cohen,University of Nebraska-Lincoln,United States,http://cse.unl.edu/~myra/,5,0,1,1,0,4,2,1,4,0,4,1,1,24
327,5,328,Dimitris E. Simos,SBA Research,Austria,https://www.sba-research.org/team/key-research...,0,0,0,0,0,0,0,0,0,0,1,1,21,23
237,6,238,Changhai Nie,Nanjing University,China,http://gist.nju.edu.cn/~changhai,5,1,0,0,3,1,0,0,0,0,1,0,5,16
431,6,432,Angelo Gargantini,University of Bergamo,Italy,http://cs.unibg.it/gargantini/,0,0,0,1,0,0,0,0,0,0,0,1,14,16
168,8,169,Rachel Tzoref-Brill,IBM Research - Haifa,Israel,http://researcher.watson.ibm.com/researcher/vi...,0,0,0,0,0,4,1,1,1,0,0,1,6,14
239,9,240,Huayao Wu,Nanjing University,China,https://huayaow.github.io/,4,0,0,0,3,1,0,0,0,0,1,0,4,13
318,9,319,Cemal Yilmaz,Sabanci University,Turkey,http://people.sabanciuniv.edu/cyilmaz/Cemal_Yi...,5,0,0,1,0,0,0,0,1,0,1,0,5,13


In [10]:
# write data to csv file
scholar.to_csv(output_rank_file, sep=',', encoding='utf-8', index=False, header=True)